In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-371606-9e78aba82a03.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

            
#date = 0 #当日は0、前日は1・・・
#
date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 18007011 # 店舗ID
SPREADSHEET_KEY = '197q3LOoYlNpTi-sxyJfOWJTD07_avNfa_3yY7YHv3UE' # スプレッドシートのID
pstart = 80 # パチンコの1台目
sstart = 1001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p50row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p50table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "50":
                    p50table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "50":
                        p50table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "250":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "250":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.5RAW'!F" + str(int(p50row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p50table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S2.5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025544', '80', '18205.0', '-9328.0', '16', '4']
['08/19', '025754', '81', '3333.0', '-2106.0', '1', '1']
['08/19', '025792', '82', '3590.0', '-3309.0', '0', '0']
['08/19', '025409', '83', '20769.0', '-5416.0', '35', '4']
['08/19', '025514', '84', '4359.0', '-1504.0', '4', '1']
['08/19', '025557', '85', '769.0', '-902.0', '0', '0']
['08/19', '025557', '86', '0', '0', '0', '0']
['08/19', '025644', '87', '17692.0', '-6920.0', '17', '5']
['08/19', '025830', '88', '3333.0', '-2406.0', '1', '1']
['08/19', '025672', '89', '0', '0', '0', '0']
['08/19', '025404', '90', '21026.0', '-9328.0', '10', '6']
['08/19', '025404', '91', '6154.0', '-1504.0', '6', '2']
['08/19', '025483', '92', '3333.0', '-2406.0', '0', '0']
['08/19', '025764', '93', '10256.0', '904.0', '6', '1']
['08/19', '025364', '94', '7436.0', '-3610.0', '3', '1']
['08/19', '025364', '95', '6410.0', '-3911.0', '2', '1']
['08/19', '025364', '96', '0', '0', '0', '0']
['08/19', '025404', '97', '3590.0', '-1203.0', '4', '2']
['

['08/19', '025579', '230', '2308.0', '-1805.0', '0', '0']
['08/19', '025579', '231', '513.0', '-1203.0', '0', '0']
['08/19', '025562', '232', '3846.0', '-1203.0', '2', '1']
['08/19', '025562', '233', '5128.0', '3913.0', '8', '1']
['08/19', '025636', '234', '0', '0', '0', '0']
['08/19', '025636', '235', '1538.0', '1205.0', '5', '1']
['08/19', '025636', '236', '0', '0', '0', '0']
['08/19', '025553', '237', '0', '0', '0', '0']
['08/19', '025553', '238', '0', '0', '0', '0']
['08/19', '025553', '239', '0', '0', '0', '0']
['08/19', '025553', '240', '0', '0', '0', '0']
['08/19', '025817', '241', '25385.0', '4515.0', '82', '11']
['08/19', '025817', '242', '0', '0', '0', '0']
['08/19', '025605', '243', '769.0', '-1203.0', '0', '0']
['08/19', '025605', '244', '7692.0', '-1504.0', '5', '3']
['08/19', '025605', '245', '4615.0', '-3008.0', '1', '1']
['08/19', '025847', '246', '2564.0', '904.0', '14', '1']
['08/19', '025802', '247', '0', '0', '0', '0']
['08/19', '025466', '248', '11026.0', '603.0', 

['08/19', '025780', '651', '3333.0', '-1203.0', '5', '1']
['08/19', '025544', '652', '11538.0', '-5416.0', '12', '4']
['08/19', '025601', '653', '9231.0', '-1805.0', '8', '4']
['08/19', '025572', '654', '769.0', '1806.0', '4', '1']
['08/19', '025694', '655', '12051.0', '-3008.0', '8', '5']
['08/19', '025445', '656', '0', '0', '0', '0']
['08/19', '025529', '657', '5897.0', '5117.0', '19', '3']
['08/19', '025235', '658', '11026.0', '6320.0', '27', '3']
['08/19', '025417', '659', '5128.0', '-3008.0', '2', '2']
['08/19', '025706', '660', '0', '0', '0', '0']
['08/19', '025705', '661', '5128.0', '-1203.0', '4', '2']
['08/19', '025121', '662', '3846.0', '5418.0', '12', '2']
['08/19', '025564', '663', '20256.0', '-1203.0', '51', '15']
['08/19', '025743', '664', '6410.0', '904.0', '9', '3']
['08/19', '025455', '665', '0', '0', '0', '0']
['08/19', '025705', '666', '7436.0', '904.0', '9', '2']
['08/19', '025381', '667', '2051.0', '-1504.0', '1', '1']
['08/19', '025756', '668', '6410.0', '-1805.0'

['08/19', '025665', '802', '35385.0', '-16249.0', '9', '2']
['08/19', '025665', '803', '24103.0', '48450.0', '45', '5']
['08/19', '025665', '804', '15641.0', '-12638.0', '1', '1']
['08/19', '025665', '805', '7179.0', '-6319.0', '0', '0']
['08/19', '025665', '806', '13333.0', '-4212.0', '6', '3']
['08/19', '025665', '807', '19231.0', '603.0', '12', '3']
['08/19', '025665', '808', '25385.0', '603.0', '15', '4']
['08/19', '025606', '809', '24872.0', '-10832.0', '6', '2']
['08/19', '025606', '810', '23846.0', '904.0', '14', '4']
['08/19', '025606', '811', '37436.0', '43334.0', '49', '5']
['08/19', '025826', '812', '15641.0', '904.0', '9', '6']
['08/19', '025826', '813', '4103.0', '-3610.0', '0', '0']
['08/19', '025826', '814', '7179.0', '7524.0', '9', '1']
['08/19', '025677', '815', '31282.0', '6621.0', '24', '5']
['08/19', '025677', '816', '34359.0', '19260.0', '33', '4']
['08/19', '025677', '817', '22308.0', '-16851.0', '1', '1']
['08/19', '025677', '818', '34872.0', '-23471.0', '4', '2'

['08/19', '120138', '1127', '0', '0', '0', '0', '--']
['08/19', '120138', '1128', '4992', '602.0', '0', '9', '20']
['08/19', '120138', '1129', '0', '0', '0', '0', '--']
['08/19', '120138', '1130', '496', '-324.0', '0', '1', '3']
['08/19', '120142', '1131', '7432', '4954.0', '0', '19', '73']
['08/19', '120142', '1132', '7849', '6574.0', '0', '22', '85']
['08/19', '120142', '1133', '2890', '-787.0', '0', '5', '30']
['08/19', '120142', '1134', '6393', '-1296.0', '0', '15', '53']
['08/19', '120130', '1135', '2713', '-2037.0', '0', '5', '9']
['08/19', '120130', '1136', '4967', '-1157.0', '0', '9', '19']
['08/19', '120130', '1137', '2727', '-1343.0', '0', '5', '10']
['08/19', '120130', '1138', '2343', '-2176.0', '0', '4', '9']
['08/19', '120104', '1139', '398', '-370.0', '0', '0', '1']
['08/19', '120104', '1140', '1497', '1389.0', '0', '4', '7']
['08/19', '120104', '1141', '412', '-185.0', '0', '1', '2']
['08/19', '120104', '1142', '1491', '185.0', '0', '3', '8']
['08/19', '120104', '1143', 

['08/19', '120115', '1264', '1848', '-1250.0', '0', '3', '15']
['08/19', '120115', '1265', '1742', '-1343.0', '0', '2', '12']
['08/19', '120115', '1266', '2849', '-694.0', '0', '7', '37']
['08/19', '119795', '1267', '2853', '-93.0', '0', '11', '16']
['08/19', '119961', '1268', '1728', '-509.0', '0', '4', '13']
['08/19', '119910', '1269', '3196', '602.0', '21', '3', '--']
['08/19', '120079', '1270', '2380', '1296.0', '0', '2', '12']
['08/19', '120079', '1271', '2083', '-1389.0', '0', '0', '10']
['08/19', '119988', '1272', '1195', '-556.0', '0', '4', '1']
['08/19', '119988', '1273', '3494', '-417.0', '0', '5', '1']
['08/19', '119993', '1274', '5829', '-1481.0', '0', '22', '40']
['08/19', '119993', '1275', '2904', '556.0', '0', '11', '25']
['08/19', '119982', '1276', '3602', '2083.0', '22', '16', '--']
['08/19', '119912', '1277', '2072', '1296.0', '14', '3', '--']
['08/19', '119912', '1278', '894', '-556.0', '1', '2', '--']
['08/19', '119997', '1279', '2694', '-556.0', '8', '8', '--']
['0

{'spreadsheetId': '197q3LOoYlNpTi-sxyJfOWJTD07_avNfa_3yY7YHv3UE',
 'updates': {'spreadsheetId': '197q3LOoYlNpTi-sxyJfOWJTD07_avNfa_3yY7YHv3UE',
  'updatedRange': 'S2.5RAW!F1226:M1297',
  'updatedRows': 72,
  'updatedColumns': 8,
  'updatedCells': 576}}

In [40]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p50table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "50":
                    p50table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "50":
                        p50table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "250":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "250":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.5RAW'!F" + str(int(p50row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p50table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S2.5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/08', '025544', '80', '12821.0', '-2106.0', '30', '6']
['08/08', '025754', '81', '3333.0', '-3008.0', '0', '0']
['08/08', '025792', '82', '0', '0', '0', '0']
['08/08', '025409', '83', '18974.0', '-5416.0', '24', '12']
['08/08', '025514', '84', '21795.0', '2107.0', '27', '7']
['08/08', '025557', '85', '0', '0', '0', '0']
['08/08', '025557', '86', '513.0', '-902.0', '0', '0']
['08/08', '025644', '87', '13590.0', '-8124.0', '6', '4']
['08/08', '025830', '88', '513.0', '-902.0', '0', '0']
['08/08', '025672', '89', '0', '0', '0', '0']
['08/08', '025404', '90', '12564.0', '-3008.0', '9', '5']
['08/08', '025404', '91', '13077.0', '-5717.0', '6', '2']
['08/08', '025483', '92', '7179.0', '14445.0', '15', '4']
['08/08', '025764', '93', '6410.0', '-5717.0', '0', '0']
['08/08', '025364', '94', '23333.0', '-2106.0', '26', '8']
['08/08', '025364', '95', '27179.0', '3612.0', '39', '11']
['08/08', '025364', '96', '2051.0', '-1805.0', '0', '0']
['08/08', '025404', '97', '0', '0', '0', '0']
['08/08'

['08/08', '025579', '229', '0', '0', '0', '0']
['08/08', '025579', '230', '0', '0', '0', '0']
['08/08', '025579', '231', '513.0', '-902.0', '0', '0']
['08/08', '025562', '232', '0', '0', '0', '0']
['08/08', '025562', '233', '0', '0', '0', '0']
['08/08', '025636', '234', '0', '0', '0', '0']
['08/08', '025636', '235', '2564.0', '-1203.0', '1', '1']
['08/08', '025636', '236', '1538.0', '-1203.0', '0', '0']
['08/08', '025553', '237', '12821.0', '1205.0', '11', '3']
['08/08', '025553', '238', '12051.0', '-5717.0', '4', '1']
['08/08', '025553', '239', '3077.0', '904.0', '2', '1']
['08/08', '025553', '240', '3846.0', '-3309.0', '0', '0']
['08/08', '025817', '241', '4103.0', '-2707.0', '1', '1']
['08/08', '025817', '242', '3077.0', '-2406.0', '0', '0']
['08/08', '025605', '243', '0', '0', '0', '0']
['08/08', '025605', '244', '20000.0', '-1805.0', '16', '6']
['08/08', '025605', '245', '0', '0', '0', '0']
['08/08', '025847', '246', '8974.0', '-902.0', '33', '4']
['08/08', '025802', '247', '0', '

['08/08', '025544', '652', '3077.0', '7825.0', '32', '2']
['08/08', '025601', '653', '6667.0', '-3309.0', '3', '2']
['08/08', '025572', '654', '1282.0', '-1203.0', '0', '0']
['08/08', '025694', '655', '7179.0', '-2406.0', '6', '3']
['08/08', '025445', '656', '1795.0', '-902.0', '1', '1']
['08/08', '025529', '657', '1282.0', '1205.0', '5', '2']
['08/08', '025235', '658', '9487.0', '1205.0', '16', '4']
['08/08', '025417', '659', '1795.0', '-1504.0', '0', '0']
['08/08', '025706', '660', '1538.0', '-1805.0', '1', '1']
['08/08', '025705', '661', '4103.0', '-2406.0', '2', '2']
['08/08', '025121', '662', '769.0', '-902.0', '0', '0']
['08/08', '025564', '663', '9231.0', '-4212.0', '10', '9']
['08/08', '025743', '664', '1282.0', '-902.0', '1', '1']
['08/08', '025455', '665', '4103.0', '-3008.0', '1', '1']
['08/08', '025705', '666', '0', '0', '1', '1']
['08/08', '025381', '667', '3590.0', '-1203.0', '3', '1']
['08/08', '025756', '668', '2821.0', '3010.0', '10', '3']
['08/08', '025760', '669', '1

['08/08', '025665', '806', '12308.0', '-9629.0', '1', '1']
['08/08', '025665', '807', '6410.0', '27987.0', '22', '1']
['08/08', '025665', '808', '18718.0', '302.0', '11', '3']
['08/08', '025606', '809', '10000.0', '-1504.0', '6', '3']
['08/08', '025606', '810', '8974.0', '-5717.0', '3', '3']
['08/08', '025606', '811', '2564.0', '-1805.0', '0', '0']
['08/08', '025826', '812', '51795.0', '-23471.0', '12', '6']
['08/08', '025826', '813', '45641.0', '-4513.0', '22', '6']
['08/08', '025826', '814', '40000.0', '-4814.0', '19', '9']
['08/08', '025677', '815', '0', '0', '0', '0']
['08/08', '025677', '816', '4615.0', '904.0', '2', '1']
['08/08', '025677', '817', '1795.0', '7825.0', '7', '1']
['08/08', '025677', '818', '15385.0', '15348.0', '18', '6']
['08/08', '119883', '1001', '916', '-417.0', '2', '0', '3']
['08/08', '119883', '1002', '2064', '-324.0', '5', '0', '7']
['08/08', '119781', '1003', '1860', '-602.0', '0', '4', '4']
['08/08', '119781', '1004', '603', '509.0', '0', '2', '5']
['08/08

['08/08', '120130', '1137', '0', '0', '0', '0', '--']
['08/08', '120130', '1138', '4109', '1296.0', '0', '4', '19']
['08/08', '120104', '1139', '0', '0', '0', '0', '--']
['08/08', '120104', '1140', '0', '0', '0', '0', '--']
['08/08', '120104', '1141', '0', '0', '0', '0', '--']
['08/08', '120104', '1142', '0', '0', '0', '0', '--']
['08/08', '120104', '1143', '90', '-139.0', '0', '0', '--']
['08/08', '120104', '1144', '0', '0', '0', '0', '--']
['08/08', '120129', '1145', '3211', '2685.0', '0', '10', '32']
['08/08', '120129', '1146', '1070', '-93.0', '0', '4', '9']
['08/08', '120129', '1147', '1277', '-741.0', '0', '5', '6']
['08/08', '120129', '1148', '1922', '2731.0', '0', '4', '29']
['08/08', '120129', '1149', '368', '93.0', '0', '1', '3']
['08/08', '120103', '1150', '1931', '-1944.0', '2', '0', '4']
['08/08', '120103', '1151', '0', '0', '0', '0', '--']
['08/08', '120103', '1152', '2144', '-231.0', '12', '2', '7']
['08/08', '120103', '1153', '1615', '-1944.0', '1', '2', '5']
['08/08', 

['08/08', '119929', '1278', '1725', '324.0', '0', '6', '9']
['08/08', '119997', '1279', '4128', '-1065.0', '11', '8', '--']
['08/08', '119997', '1280', '5087', '-2315.0', '12', '10', '--']
['08/08', '120033', '1281', '5642', '4907.0', '33', '12', '--']
['08/08', '120033', '1282', '5074', '231.0', '17', '12', '--']
['08/08', '120033', '1283', '4413', '-1806.0', '10', '8', '--']
['08/08', '120102', '1292', '325', '231.0', '0', '1', '6']
['08/08', '120102', '1293', '0', '0', '0', '0', '--']
['08/08', '119949', '1294', '2591', '-2037.0', '8', '5', '--']
['08/08', '119949', '1295', '540', '-556.0', '0', '1', '--']
['08/08', '119891', '1296', '164', '-139.0', '0', '4', '--']
['08/08', '119891', '1297', '0', '0', '0', '0', '--']
['08/08', '120021', '1298', '4898', '-1250.0', '27', '13', '--']
['08/08', '120021', '1299', '3164', '-1713.0', '13', '9', '--']
['08/08', '119913', '1300', '1464', '-1111.0', '1', '1', '--']
['08/08', '119913', '1301', '880', '648.0', '6', '4', '--']
['08/08', '11991

{'spreadsheetId': '197q3LOoYlNpTi-sxyJfOWJTD07_avNfa_3yY7YHv3UE',
 'updates': {'spreadsheetId': '197q3LOoYlNpTi-sxyJfOWJTD07_avNfa_3yY7YHv3UE',
  'updatedRange': 'S2.5RAW!F506:M577',
  'updatedRows': 72,
  'updatedColumns': 8,
  'updatedCells': 576}}